In [313]:
import pymongo
client = pymongo.MongoClient(host='120.79.139.239', port=27017)
db = client['weibo']

# User Collection

In [ ]:
import re, sys, bson
collection = db['users']
regex = re.compile('(?:谈股|财经|说股|期货|外汇|讲股|论股|看股|港股|美股|理财师|A股|经济政策|短线|炒股|荐股|盈亏|炒股|股利|股海|股民|股票|个股|做空|诱多|诱空|踏空|长空|短空|长多|短多|死多|翻多|翻空|多杀多|扎空|空仓|建仓|满仓|斩仓|减仓|加仓|重仓|清仓|套牢|补仓|仓位|庄家|震仓|追涨|杀跌|止盈|抄底|逃顶|盘整|回档|坐庄|吸筹|对敲|洗盘|散户|中户|坐轿|抬轿|筹码|抢帽子|多头陷阱|空头陷阱|护盘|跳空|开盘价|收盘价|最高价|最低价|成交量|放量|缩量|热门股|冷门股|白马股|黑马股|龙头股|阴跌|换手率|现手|平开|低开|高开|内盘|外盘|均价|浮筹|市盈率|含权|回购|基本面分析|量比|每股|收益|市净率|探底|填权|停牌|退市)')    
nregex = re.compile('(?:公司|企业|CEO|基金会|大学|学院|政府)')

weibo_list = collection.find({}, {'id': 1, 'description': 1})

In [ ]:
print (weibo_list.count())

In [ ]:
id_del = []
for x in weibo_list:
    desp = x['description']
    if re.search(nregex, desp) != None or re.search(regex, desp) == None:
        id_del.append(x['id'])

for id in id_del:
    collection.delete_one({'id': id})

In [ ]:
results = collection.find({}, {'id': 1})
id_list = []
for result in results:
    id_list.append(str(result['id']))
print (len(id_list))

In [2]:
collection = db['weibos']
results = collection.find({}, {'created_at': 1})
time_list = []
import re
for result in results:
    if re.search('2018-07-20', result['created_at']) != None:
        time_list.append(str(result['created_at']))
time_list = sorted(time_list, reverse=True)
print (len(time_list))
print (time_list)

2007
['2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00:00', '2018-07-20 00

False

In [317]:
import time
date = time.strftime('%Y-%m-%d', time.localtime())
ldate = time.strftime('%Y-%m-%d', time.localtime(time.time() - 24 * 60 * 60))
lldate = time.strftime('%Y-%m-%d', time.localtime(time.time() - 2 * 24 * 60 * 60))
print (date)
print (ldate)
print (lldate)
h = "{}|{}|{}".format(date, ldate, lldate)
results1 = collection.find({'created_date': {'$regex': date}})
results2 = collection.find({'created_date': {'$regex': ldate}})
print (results1.count())
print (results2.count())

# weibo_id_list = []
# for delete in deletes: 
#     weibo_id_list.append(delete['id'])
# print (len(weibo_id_list))

2018-07-23
2018-07-22
2018-07-21
886
1683


/home/impulse/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  # This is added back by InteractiveShellApp.init_path()
/home/impulse/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  if sys.path[0] == '':


In [ ]:
print (weibo_id_list)

# Analyzing

## Load weibo_test

In [295]:
import pymongo
import pandas as pd
def get_data():
    client = pymongo.MongoClient(host='120.79.139.239', port=27017)
    db = client['weibo']
    collection = db['weibos']
    results = collection.find({}, {'user':1,'attitudes_count':1,'comments_count':1,'reposts_count':1,'text':1,'full_text':1})
    data = pd.DataFrame(list(results))
    if 'attitudes_count' in data.columns: ## if it is normal
        data = data.get(['user','attitudes_count','comments_count','reposts_count','text', 'full_text'])
        data.columns = ['user', 'at_cnt', 'cmt_cnt', 'rep_cnt', 'text', 'f_text']
        for i in range(len(data)):
            if len(str(data.loc[i, 'f_text'])) > 135:
                data.loc[i, 'text'] = data.loc[i, 'f_text']
        data = data.drop('f_text', axis=1)
    else:
        print ('some errors occur')
        
    return data

In [296]:
import re

def read_stop_words(): 
    stop_words = []
    stop_letters = []
    with open('stop_words.txt') as sp:
        for x in sp:
            stop_words.append(x[:-1])
    with open('stop_letters.txt') as sp:
        for x in sp:
            stop_letters.append(x[:-1])
    stop_words = set(stop_words)
    re_stop_letter = re.compile('|'.join(stop_letters))
    return (stop_words, re_stop_letter)

In [297]:
import re, jieba
jieba.load_userdict('jieba_dict_companys')

def get_clean_text(data, output=True):
    texts = []
    for cnt, raw in enumerate(data.text):
        del_name = re.findall('@([^ |<|:|\(|\)|\\|\/|<|>|\[|\]]+)', raw)
        for name in del_name:
            raw = re.sub(name, '', raw)
        raw = re.sub('<br />', '，', raw)
        raw = re.sub('[^\u4e00-\u9fa5|，|!|。]+|\?{1}|:{1}', '', raw)
        texts.append(raw)
        
        if cnt % 1000 == 0 and output == True:
            print ('cnt = {}'.format(cnt))
    if len(texts) == len(data):
        return pd.Series(texts)
    else:
        print('not the same length')
        return None

In [298]:
import re, jieba
import jieba.posseg as postag
                           
def get_words(data, re_stop_letter, stop_words, output=True):
    words = []
    for cnt, raw in enumerate(data.c_text):
        result = postag.cut(raw)
        raw = [x.word for x in result if (len(x.word) > 1 and 'n' in x.flag and re.search(re_stop_letter, x.word) == None and x.word not in stop_words)]
        words.append(raw)
        
        if cnt % 1000 == 0 and output == True:
            print ('cnt = {}'.format(cnt))
            
    return pd.Series(words)

In [319]:
def get_word_count(lst):
    count = {}
    for x in lst:
        if x in count:
            count[x] += 1
        else:
            count[x] = 1
    return count

# sentiments

In [300]:
import requests, re, time, json

def get_sentiment(data, output=True):
    headers = {
        'Content-Type': 'application/json'
    }
    params = {
        'access_token': '24.cb5b8cccad6a49c21d4cccd1a047f9ae.2592000.1534940191.282335-11569351'
    }
    positive_prob = []
    for cnt, text in enumerate(data.c_text):
        start_time = time.clock()
        post_json = {
            "text": text
        }

        response = requests.post('https://aip.baidubce.com/rpc/2.0/nlp/v1/sentiment_classify',
                                params=params, headers=headers, json=post_json)
        if response.status_code != 200:
            time.sleep(2.0)
            response = requests.post('https://aip.baidubce.com/rpc/2.0/nlp/v1/sentiment_classify',
                                params=params, headers=headers, json=post_json)

        if response.text != None:
            res_json = json.loads(response.text)
            if res_json.get('items') != None and res_json.get('items')[0].get('positive_prob') != None: 
                prob = res_json.get('items')[0].get('positive_prob')
                #print (prob)
                positive_prob.append(prob)
            else:
                print('-1')
                positive_prob.append('-1')
        else:
            print('-1')
            positive_prob.append('-1')
        elapsed = (time.clock() - start_time)
        time.sleep(0.25 - elapsed)

        if cnt % 100 == 0 and output:
            print ('cnt = {}'.format(cnt))
    if len(data) != len(positive_prob):
        print ('length not equal')
        
    return pd.Series(positive_prob)
    

In [301]:
data = get_data()

In [302]:
stop_words, re_stop_letter = read_stop_words()

In [303]:
data['c_text'] = get_clean_text(data)

cnt = 0
cnt = 1000
cnt = 2000
cnt = 3000
cnt = 4000
cnt = 5000


In [515]:
data['words'] = get_words(data=data, re_stop_letter=re_stop_letter, stop_words=stop_words)

cnt = 0
cnt = 1000
cnt = 2000
cnt = 3000
cnt = 4000
cnt = 5000


In [ ]:
# notice ! in use !
# data['senti'] = get_sentiment(data)

In [516]:
word_counts = [ get_word_count(lst) for lst in data.words]
data['dict'] = pd.Series(word_counts)  

In [935]:
data['level'] = (1 + data.at_cnt) * (1 + data.rep_cnt) * (1 + data.cmt_cnt)

In [938]:
#data.to_csv('../ScrapyDatas/weibo_test_data.csv')

## load idf

In [348]:
import math, numpy as np
def tf(word, count):
    return count[word] / sum(count.values())

def n_containing(word, count_list):
    return sum(1 for count in count_list if word in count)
    
def idf(word, count_list):
    return math.log(len(count_list) / (1 + n_containing(word, count_list)))

def tfidf(word, count, count_list):
    return tf(word, count) * idf(word, count_list)

In [345]:
data_idf_dict = pd.read_csv('../ScrapyDatas/weibo_idf_dict.csv', header=None)

In [362]:
data_idf_dict = [ eval(dic) for dic in np.array(data_idf_dict[1]).tolist() ]

In [517]:
data_dict_all = {}
for dic in data.dict:
    for k, v in dic.items():
        if k in data_dict_all:
            data_dict_all[k] += v
        else:
            data_dict_all[k] = v
            
        if len(k) == 4:
            head, tail = k[2:], k[:-2]
            if head in data_dict_all:
                data_dict_all[head] += v
            else:
                data_dict_all[head] = v
                
            if tail in data_dict_all:
                data_dict_all[tail] += v
            else:
                data_dict_all[tail] = v

In [518]:
# remove not nouns

from snownlp import SnowNLP
def tpok(word):
    string = str(word)
    s = SnowNLP(string)
    ls = list(s.tags)
    if len(ls) >= 3:
        return False
    if len(ls) == 1 and 'n' not in ls[0][1]:
        return False
    if len(word) == 2:
        tail = str(word[-1])
        st = SnowNLP(tail)
        ls = list(st.tags)
        if len(ls) == 1 and ls[0][1] == 'v':
            return False
    return True

data_dict_all = dict([ (k, v) for k, v in data_dict_all.items() if tpok(k)])

In [519]:
data_dict_all = dict(sorted(data_dict_all.items(), key=lambda x: math.sqrt(len(x[0]))*x[1], reverse=True))
data_dict_all = dict([(x,y) for x,y in data_dict_all.items() if y > 3])

In [520]:
data_words_list_clean = []
for dic in data.dict:
    data_words_clean = {}
    for k, v in dic.items():
        if k in data_dict_all:
            data_words_clean[k] = v
    data_words_list_clean.append(data_words_clean)

data.dict = data_words_list_clean

### have dict for each, get dict for all

In [367]:
data_idf_dict_all = {}
for dic in data_idf_dict:
    for k, v in dic.items():
        if k in data_idf_dict_all:
            data_idf_dict_all[k] += v
        else:
            data_idf_dict_all[k] = v
            
        if len(k) == 4:
            head, tail = k[2:], k[:-2]
            if head in data_idf_dict_all:
                data_idf_dict_all[head] += v
            else:
                data_idf_dict_all[head] = v
                
            if tail in data_idf_dict_all:
                data_idf_dict_all[tail] += v
            else:
                data_idf_dict_all[tail] = v

In [430]:
data_idf_dict_all = dict(sorted(data_dict_all.items(), key=lambda x: math.sqrt(len(x[0]))*x[1], reverse=True))
data_idf_dict_all = dict([(x,y) for x,y in data_dict_all.items() if y > 10])

### work idf

### 关键词

In [490]:
# only tf for all
print("Top words in all documents")
all_scores = { word: tf(word, data_dict_all) for word in data_dict_all}
all_sorted_words = sorted(all_scores.items(), key=lambda x: math.sqrt(len(x[0]))*x[1], reverse=True)
for word, score in all_sorted_words[:100]:
    print("\tWord: {}, TF: {}".format(word, round(score, 5))) 

Top words in all documents
	Word: 疫苗, TF: 0.09456
	Word: 生产, TF: 0.02245
	Word: 国家药监局, TF: 0.00913
	Word: 医药, TF: 0.0097
	Word: 创业板, TF: 0.00757
	Word: 狂犬病, TF: 0.00669
	Word: 李克强, TF: 0.00651
	Word: 概念股, TF: 0.00619
	Word: 药品, TF: 0.00599
	Word: 软件, TF: 0.00525
	Word: 生物, TF: 0.00516
	Word: 军工, TF: 0.00466
	Word: 小米, TF: 0.00414
	Word: 新浪, TF: 0.00407
	Word: 石墨, TF: 0.00407
	Word: 黄金, TF: 0.00392
	Word: 龙头, TF: 0.00387
	Word: 康泰, TF: 0.00382
	Word: 原油, TF: 0.00377
	Word: 电解锰, TF: 0.00296
	Word: 科技股, TF: 0.00291
	Word: 银行, TF: 0.00343
	Word: 消息面, TF: 0.00279
	Word: 特朗普, TF: 0.00276
	Word: 食品, TF: 0.00331
	Word: 金融, TF: 0.00328
	Word: 进口, TF: 0.00321
	Word: 金融股, TF: 0.00259
	Word: 道德, TF: 0.00308
	Word: 中兴通讯, TF: 0.00217
	Word: 涉嫌犯罪, TF: 0.00215
	Word: 产品, TF: 0.00301
	Word: 狂犬, TF: 0.00296
	Word: 贸易战, TF: 0.00239
	Word: 茅台, TF: 0.00289
	Word: 国务院, TF: 0.00232
	Word: 通讯, TF: 0.00284
	Word: 医疗, TF: 0.00276
	Word: 腾讯, TF: 0.00276
	Word: 航天, TF: 0.00276
	Word: 电子, TF: 0.00271
	Word: 犯罪, TF

In [603]:
# tf-idf for all
print("Top words in all documents")

all_scores = { word : tfidf(word, data_dict_all, data_idf_dict_all)
                     for word in data_dict_all }
all_sorted_words = sorted(all_scores.items(), key=lambda x: math.sqrt(len(x[0]))*x[1], reverse=True)
for word, score in all_sorted_words[:100]:
    print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5))) 

Top words in all documents
	Word: 疫苗, TF-IDF: 0.60041
	Word: 创业板, TF-IDF: 0.05394
	Word: 医药, TF-IDF: 0.06035
	Word: 狂犬病, TF-IDF: 0.04761
	Word: 李克强, TF-IDF: 0.04639
	Word: 概念股, TF-IDF: 0.0441
	Word: 软件, TF-IDF: 0.03795
	Word: 药品, TF-IDF: 0.03657
	Word: 军工, TF-IDF: 0.03321
	Word: 生物, TF-IDF: 0.03084
	Word: 小米, TF-IDF: 0.02952
	Word: 新浪, TF-IDF: 0.02899
	Word: 石墨, TF-IDF: 0.02899
	Word: 黄金, TF-IDF: 0.02846
	Word: 龙头, TF-IDF: 0.02759
	Word: 康泰, TF-IDF: 0.02723
	Word: 原油, TF-IDF: 0.02518
	Word: 消息面, TF-IDF: 0.01985
	Word: 特朗普, TF-IDF: 0.01968
	Word: 进口, TF-IDF: 0.0239
	Word: 科技股, TF-IDF: 0.01942
	Word: 道德, TF-IDF: 0.02337
	Word: 金融股, TF-IDF: 0.01845
	Word: 食品, TF-IDF: 0.02205
	Word: 中兴通讯, TF-IDF: 0.01546
	Word: 涉嫌犯罪, TF-IDF: 0.01529
	Word: 银行, TF-IDF: 0.02124
	Word: 贸易战, TF-IDF: 0.01704
	Word: 茅台, TF-IDF: 0.02056
	Word: 国务院, TF-IDF: 0.01652
	Word: 医疗, TF-IDF: 0.01985
	Word: 狂犬, TF-IDF: 0.01975
	Word: 腾讯, TF-IDF: 0.01968
	Word: 航天, TF-IDF: 0.01968
	Word: 通讯, TF-IDF: 0.01892
	Word: 产品, TF-ID

In [604]:
### 输出关键词
## TODO

In [523]:
# tf-idf for each

def get_critical_words(data):
    countlist = data.dict
    critical_words = []
    for i, count in enumerate(data.dict):
        #print("Top words in document {}".format(i))
        scores = {word: tfidf(word, count, data_idf_dict) for word in count}
        sorted_words = sorted(scores.items(), key=lambda x: math.sqrt(len(x[0]))*x[1], reverse=True)
        critical_word = []
        for word, score in sorted_words[:3]:
            critical_word.append(word)
            #print("\tWord: {}, TF-IDF: {}".format(word, round(score, 3))) 
        critical_words.append(critical_word)
        if i % 100 == 0:
            print ('cnt = {}'.format(i))
    return critical_words

In [526]:
critical_words_list = get_critical_words(data)

cnt = 0
cnt = 100
cnt = 200
cnt = 300
cnt = 400
cnt = 500
cnt = 600
cnt = 700
cnt = 800
cnt = 900
cnt = 1000
cnt = 1100
cnt = 1200
cnt = 1300
cnt = 1400
cnt = 1500
cnt = 1600
cnt = 1700
cnt = 1800
cnt = 1900
cnt = 2000
cnt = 2100
cnt = 2200
cnt = 2300
cnt = 2400
cnt = 2500
cnt = 2600
cnt = 2700
cnt = 2800
cnt = 2900
cnt = 3000
cnt = 3100
cnt = 3200
cnt = 3300
cnt = 3400
cnt = 3500
cnt = 3600
cnt = 3700
cnt = 3800
cnt = 3900
cnt = 4000
cnt = 4100
cnt = 4200
cnt = 4300
cnt = 4400
cnt = 4500
cnt = 4600
cnt = 4700
cnt = 4800
cnt = 4900
cnt = 5000
cnt = 5100
cnt = 5200


In [529]:
data['critical_word'] = critical_words_list

In [532]:
#data.to_csv('../ScrapyDatas/weibo_test_data.csv')

In [588]:
word_senti = {}
for sen, cri_word in zip(data.senti, data.critical_word):
    #print(sen)
    #print (cri_word)
    for w in cri_word:
        if sen == None or not isinstance(sen, float) or sen < 0:
            continue
        if (sen > 0.7):
            if w in word_senti:
                word_senti[w] += 1
            else:
                word_senti[w] = 1
        if (sen < 0.3):
            if w in word_senti:
                word_senti[w] -= 1
            else:
                word_senti[w] = -1

### 好坏词语

In [589]:
word_senti_posi = sorted(word_senti.items(), key=lambda x: x[1], reverse=True)[:30]
word_senti_nega = sorted(word_senti.items(), key=lambda x: x[1], reverse=False)[:30]

In [598]:
for x in word_senti_posi:
    print (x)

('概念股', 32)
('卫信康', 28)
('富安娜', 27)
('东睦', 27)
('西仪', 26)
('信号', 25)
('原油', 23)
('差价', 22)
('石墨', 20)
('财经频道', 16)
('黄金', 15)
('恒银', 15)
('中国国旅', 15)
('黑石', 15)
('污水处理', 14)
('电气', 13)
('独角兽', 12)
('总体', 12)
('香港', 12)
('水稻', 12)
('袁隆平', 12)
('天才', 12)
('台风', 11)
('青海华鼎', 11)
('科研院所', 11)
('李琦', 11)
('教育', 10)
('金融股', 10)
('北非', 10)
('财长', 10)


In [599]:
for x in word_senti_nega:
    print (x)

('疫苗', -485)
('李克强', -69)
('医药', -41)
('违法者', -36)
('涉嫌犯罪', -35)
('狂犬病', -30)
('渎职', -30)
('创业板', -30)
('生物医药', -26)
('产品检验', -25)
('三聚氰胺', -25)
('集合竞价', -24)
('进程', -24)
('任由', -23)
('需谨慎', -23)
('道德', -20)
('食药监', -19)
('奶粉', -19)
('康泰', -19)
('老规矩', -18)
('监管部门', -16)
('药监局', -16)
('锂电池', -16)
('巨资', -16)
('亿万富翁', -15)
('留条', -15)
('后路', -15)
('政协提案', -15)
('食品药品', -14)
('性质', -14)


In [605]:
### 输出好坏词语
## TODO

### 词语网络

In [906]:
class Graph(object):
    def __init__(self):
        self.node_cnt = 0
        self.id = {}
        self.value = {}
        self.deg = {}
        self.name = {}
        self.edges = {}
        self.lim = 1024

In [907]:
wG = Graph()

In [908]:
for dic in data.dict:
    for x in dic.keys():
        if x not in word_id:
            wG.id[x] = wG.node_cnt
            wG.node_cnt += 1
            idx = wG.id[x]
            wG.name[idx] = x
            wG.value[idx] = 1
            wG.deg[idx] = 0
            wG.edges[idx] = {}

In [909]:
for dic in data.dict:
    for x in dic.keys():
        idx = wG.id[x]
        if idx not in wG.deg:
            wG.deg[idx] = len(dic) - 1
        else:
            wG.deg[idx] += len(dic) - 1
del_idx = set()
for idx, val in wG.deg.items():
    if val < wG.lim:
        del_idx.add(idx)
for idx in del_idx:
    wG.name.pop(idx)
    wG.deg.pop(idx)
    wG.value.pop(idx)
    wG.edges.pop(idx)
wG.id = dict([ (k, v) for k, v in wG.id.items() if v not in del_idx ])

In [910]:
for dic in data.dict:
    for x in dic.keys():
        if x in wG.id:
            idx = wG.id[x]
            for y in dic.keys():
                if y != x and y in wG.id:
                    idy = wG.id[y]
                    if idx not in wG.edges[idy]:
                        wG.edges[idx][idy] = 1
                        wG.edges[idy][idx] = 1
                    else:
                        wG.edges[idx][idy] += 1
                        wG.edges[idy][idx] += 1

In [911]:
wG.edges = dict(sorted([ (k, v) for k, v in wG.edges.items() if len(v) > 1 ], key = lambda x: -len(x[1])))

In [912]:
for x in [ {wG.name[idx]: { wG.name[idy]: v for idy, v in chx.items() if idy in wG.name} } for idx, chx in wG.edges.items()] :
    print (x)

{'军工': {'阿里': 92, '腾讯': 94, '概念股': 82, '发动机': 98, '叶片': 90, '石墨': 98, '空白': 90, '电子': 18, '疫苗': 20, '进口': 22, '医药': 14, '狂犬病': 2, '软件': 8, '工程': 20, '新华': 4, '创业板': 14, '药品': 4}}
{'疫苗': {'医药': 216, '狂犬病': 292, '药品': 204, '新华': 32, '软件': 62, '进口': 86, '创业板': 80, '石墨': 14, '阿里': 4, '腾讯': 16, '概念股': 40, '空白': 6, '军工': 20, '电子': 6, '工程': 6, '发动机': 2, '叶片': 2}}
{'腾讯': {'阿里': 118, '概念股': 74, '军工': 94, '发动机': 94, '叶片': 94, '石墨': 96, '空白': 94, '新华': 6, '软件': 2, '疫苗': 16, '医药': 8, '狂犬病': 2, '工程': 20, '进口': 18, '药品': 6, '创业板': 4}}
{'新华': {'疫苗': 32, '阿里': 6, '腾讯': 6, '软件': 42, '药品': 6, '狂犬病': 6, '电子': 72, '创业板': 40, '概念股': 42, '工程': 4, '医药': 4, '军工': 4, '发动机': 2, '叶片': 2, '石墨': 2, '空白': 2}}
{'概念股': {'阿里': 74, '腾讯': 74, '军工': 82, '发动机': 72, '叶片': 72, '石墨': 72, '空白': 72, '医药': 24, '疫苗': 40, '软件': 42, '创业板': 52, '电子': 38, '新华': 42, '药品': 4, '工程': 2}}
{'石墨': {'阿里': 96, '腾讯': 96, '概念股': 72, '军工': 98, '发动机': 114, '叶片': 110, '空白': 114, '创业板': 12, '软件': 54, '疫苗': 14, '工程': 20, '进口': 18, '医药': 4, '新华': 2,

### Top weibos

In [939]:
hottest_data = data.sort_values(by='level', axis=0, ascending=False)[:30]

In [945]:
print (hottest_data.get(['level', 'text']))

            level                                               text
3302  25493009648  要不是长生造假疫苗事件，我们都快忘记当年的孙咸泽和三鹿 ，忘记了中国奶粉曾经害惨了多少孩童，...
3648  11774105640  高俊芳   对这些丧尽天良的巨富，可不可以满门抄斩？普通百姓抓错一只鸟，坐牢十年。小老板开个...
3183   1108146984  【宣称月薪1万6，结果被骗去韩国种萝卜】广告里说去韩国打工包吃包住月薪16000元，结果去了...
3090    249010104  【山东查明不合格疫苗流向，涉及儿童215184人】据山东省疾控中心，长春长生公司销往山东省的...
2519    226019160  看了看生物疫苗行业平均毛利率在80%以上，最高的高达91%以上，净利率在30%-40%，研发...
2439    225661280  好好努力赚钱，多多锻炼身体，不养狗远离狗[加油]移民不了，也得给下一代花朵用进口奶粉，用进口...
2538    223518945  明天要的人，点赞！<a data-url="http://t.cn/Rg1Mmw6" hre...
2527    116772480  药监局监管部门应该协同公安，工商，税务机关马上查封所有长生产品，封存一切长生疫苗，所有批次重...
3099    113876728  【山东查明不合格疫苗流向 21万余儿童已接种长春长生公司百白破疫苗！】<a  href=\"...
2523     95465200  人命关天的事不能儿戏，长生事件反映出企业责任，商誉，良心，监管部门的监管和公信等诸多问题！这...
2525     89882272  都过了一上午了，还他妈没动静，第一步相关部门先给所有长生产品都封存，停止使用和流通！这一上午...
2434     79872272  一个人走完一生不容易，从出生到进墓地，每个阶段如同过关。儿童阶段要过：疫苗关，奶粉关，幼儿园...
2526     76330254  相关部门干嘛了，还等明天上班吗？马上查封一切长生产品！【#长生生物疫苗事件#：不能任由恐慌、...
2521     66548412  今天越来越多的大咖，明星们关注